## Multiclass classification on the Consumer complaints Dataset

**Dataset downloaded form: https://catalog.data.gov/dataset/consumer-complaint-database**

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
!find /c /v "complaints.csv"

^C


### Since the number of data points is very large, only considering first 5 million entries

In [8]:
df = pd.read_csv('complaints.csv', nrows = 1000000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Date received                 1000000 non-null  object
 1   Product                       1000000 non-null  object
 2   Sub-product                   987704 non-null   object
 3   Issue                         1000000 non-null  object
 4   Sub-issue                     875430 non-null   object
 5   Consumer complaint narrative  444832 non-null   object
 6   Company public response       489118 non-null   object
 7   Company                       1000000 non-null  object
 8   State                         983852 non-null   object
 9   ZIP code                      983758 non-null   object
 10  Tags                          117497 non-null   object
 11  Consumer consent provided?    834434 non-null   object
 12  Submitted via                 1000000 non-n

In [9]:
df = df[['Product', 'Consumer complaint narrative']]

In [10]:
df.head()

,Product,Consumer complaint narrative
0,"Credit reporting, credit repair services, or o...",NaN
1,Vehicle loan or lease,I contacted Ally on Friday XX/XX/XXXX after fa...
2,"Credit reporting, credit repair services, or o...",NaN
3,Checking or savings account,NaN
4,Credit card or prepaid card,On XX/XX/2019 my bill did not go through accou...


In [13]:
df = df.dropna(axis = 0)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444832 entries, 1 to 999998
Data columns (total 2 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Product                       444832 non-null  object
 1   Consumer complaint narrative  444832 non-null  object
dtypes: object(2)
memory usage: 10.2+ MB


In [15]:
df.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    223670
Debt collection                                                                  75666
Credit card or prepaid card                                                      42258
Mortgage                                                                         35450
Checking or savings account                                                      25313
Money transfer, virtual currency, or money service                               12426
Student loan                                                                     10312
Vehicle loan or lease                                                             9939
Payday loan, title loan, or personal loan                                         6548
Credit reporting                                                                  1047
Credit card                                                                        777
Bank account or service                    

**Some of the Products can be clubbed together like:**
- Consolidate “Credit reporting” into “Credit reporting, credit repair services, or other personal consumer reports”.
- Consolidate “Credit card” into “Credit card or prepaid card”.
- Consolidate “Payday loan” into “Payday loan, title loan, or personal loan”.
- Consolidate “Virtual currency” into “Money transfer, virtual currency, or money service”.  
“Other financial service” has very few number of complaints and it does not mean anything, so it is removed.